# CUP

### Lettura, understanding e preparazione dei dati

In [83]:
from utils import *

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer
# per visualizzare i dati in 3D
from mpl_toolkits.mplot3d import Axes3D
# per rimuovere i warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
header_tr = ['id', 'input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'input_6', 'input_7', 'input_8', 'input_9', 'input_10', 'input_11', 'input_12', 'output_x', 'output_y', 'output_z']
header_ts = ['id', 'input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'input_6', 'input_7', 'input_8', 'input_9', 'input_10', 'input_11', 'input_12']

In [ ]:
# path per fisso (TR)
training_set_path = r'C:\Users\Giovanni\Desktop\Machine Learning (ML)\new\16-ML-24-PRJ lecture amp package-20250204\ML-Working\ML_course_stuffs\ML-CUP24-TR.csv'
# path per portatile (TR)
# training_set_path = r'C:\Users\hp\Desktop\Machine Learning (ML)\ML-Working\data\ML-CUP24-TR.csv'
df_TR = pd.read_csv(training_set_path, comment='#', names=header_tr)

# path per fisso (TS)
test_set_path = r'C:\Users\Giovanni\Desktop\Machine Learning (ML)\new\16-ML-24-PRJ lecture amp package-20250204\ML-Working\ML_course_stuffs\ML-CUP24-TS.csv'
# path per portatile (TS)
# test_set_path = r'C:\Users\hp\Desktop\Machine Learning (ML)\ML-Working\data\ML-CUP24-TS.csv'
df_TS = pd.read_csv(test_set_path, comment='#', names=header_ts)

In [ ]:
print(df_TR.shape)
print(df_TS.shape)

In [ ]:
df_TR

In [ ]:
df_TS

In [ ]:
# preparazione dati
X = df_TR.iloc[:, 1:-3]
y = df_TR.iloc[:, -3:]

X_blind_test = df_TS.iloc[:, 1:]
# y_blind_test AKA blind test questo ce l'ha il prof

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
y.describe()

- la *x* delle label ha un range tra -2.2 e 2.4
- la *y* delle label ha un range tra -2.5 e 2
- la *z* delle label ha un range tra -9.7 e 25.8

In [ ]:
y

In [ ]:
# Plot lables del file di training della CUP in 3D

y_array = y.to_numpy() 

X_coor = y_array[:, 0]  
Y_coor = y_array[:, 1]  
Z_coor = y_array[:, 2] 

# Creazione figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot 3D
ax.scatter(X_coor, Y_coor, Z_coor, c=Z_coor, cmap='coolwarm', marker='o')

# Label assi
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Plot labels di training in 3D")

plt.show()

In [ ]:
X_blind_test.shape

In [ ]:
X_blind_test

In [ ]:
# Split dei dati (dev e internal test set)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training + Validation Set (Dev Set): {len(X_dev)} campioni")
print(f"Internal Test Set: {len(X_test)} campioni")

In [ ]:
print(X_dev.shape)
print(y_dev.shape)   
print(X_test.shape)
print(y_test.shape) 

In [ ]:
y_dev.describe()

In [ ]:
# Split dei dati (train e val set)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.2, random_state=42)

print(f"Training Set: {len(X_train)} campioni")
print(f"Validation Set: {len(X_val)} campioni")

In [ ]:
print(X_train.shape)
print(y_train.shape)    
print(X_val.shape)
print(y_val.shape)

### Normalizzazione dei dati

In [ ]:
# NORMALIZZAZIONE CON STANDARD SCALER
scaler_X, X_train_scaled, X_val_scaled, X_test_scaled, = StandardScalerFun(X_train, X_val, X_test)
scaler_y, y_train_scaled, y_val_scaled, y_test_scaled = StandardScalerFun(y_train, y_val, y_test)

# Normalizzazione blind
columns_blind = X_blind_test.columns
index_blind = X_blind_test.index
X_blind_test_scaled_np = scaler_X.transform(X_blind_test)
X_blind_test_scaled = pd.DataFrame(X_blind_test_scaled_np, columns=columns_blind, index=index_blind)

In [ ]:
# Ricreazione di X_dev dopo la normalizzazione
X_dev_scaled = pd.concat([X_train_scaled, X_val_scaled])
y_dev_scaled = pd.concat([y_train_scaled, y_val_scaled])

### Allenamento con grid e internal test

In [ ]:
epochs = 500

In [ ]:
# Standard scaling

# NN per task di regressione
mlp = MLPRegressor(max_iter=epochs, random_state=42,)

# Creazione dello scorer personalizzato per MEE
mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)

param_grid = [
         {
        'hidden_layer_sizes': [(3,), (50,), (5, 3), (50, 50), (100, 80), (10, 8, 5), (50, 50, 50), (200, 200, 200)],
        'activation': ['tanh', 'relu'],  
        'solver': ['sgd'], 
        'alpha': [0.0001, 0.001, 0.01, 0.1], 
        'learning_rate_init': [0.01, 0.02, 0.001, 0.002, 0.0001],
        'learning_rate': ['constant', 'adaptive'],  
        'batch_size': [16, 32, 64], # 1 -> online, 'auto' --> fullbatch, other --> minibatch
        'momentum': [0.9, 0.95]
    },
]

# Creazione del K-Fold Cross Validation (5 fold)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Creazione di GridSearchCV usando MSE come scoring per il training
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1, refit=True)

# Esecuzione Grid Search usando Training Set e Validation Set (AKA dev set)
grid_search.fit(X_dev_scaled, y_dev_scaled)

# Miglior modello trovato dalla Grid Search
best_mlp = grid_search.best_estimator_

# Calcolo del MEE medio con cross-validation sul miglior modello
mee_scores = cross_val_score(best_mlp, X_dev, y_dev, cv=cv, scoring=mee_scorer, n_jobs=-1)

# Migliori parametri trovati
print("Migliori parametri trovati:", grid_search.best_params_)
print("MSE medio SCALATO (sui 5 fold) sul Validation:", -grid_search.best_score_)
print("MEE medio SCALATO (sui 5 fold) sul Validation:", -np.mean(mee_scores))

In [ ]:
plt.figure(figsize=(12, 5))

# Grafico con scala lineare
plt.subplot(1, 2, 1)  # (1 riga, 2 colonne, primo grafico)
plt.plot(best_mlp.loss_curve_, label="MSE (Loss)", color="blue")
plt.xlabel("Epoche")
plt.ylabel("MSE")
plt.title("Loss Curve scalata (Scala Lineare)")
plt.legend()
plt.grid(True)

# Grafico con scala logaritmica
plt.subplot(1, 2, 2)  # (1 riga, 2 colonne, secondo grafico)
plt.plot(best_mlp.loss_curve_, label="MSE (Loss)", color="blue")
plt.xlabel("Epoche")
plt.ylabel("MSE")
plt.title("Loss Curve scalata (Scala Logaritmica)")
plt.yscale("log")  # Imposta scala logaritmica
plt.legend()
plt.grid(True, which="both", linestyle="--")  # Grid anche sui minori

plt.tight_layout()  # Migliora la disposizione
plt.show()

In [ ]:
# Visualizzazione miglior iperparametri
params = grid_search.best_params_
params

In [ ]:
# Creazione del dataframe contenente tutti i risultati della Grid Search
cv_res_df = pd.DataFrame(grid_search.cv_results_)

In [ ]:
cv_res_df

In [ ]:
# Valutazione sull'Internal Test Set
y_pred_test_scaled = best_mlp.predict(X_test_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1, y_test.shape[1]))

test_mse = mean_squared_error(y_test, y_pred_test)
test_mee = mean_euclidean_error(y_test, y_pred_test)

print(f"MSE sull'Internal Test Set: {test_mse:.5f}")
print(f"MEE sull'Internal Test Set: {test_mee:.5f}")

In [ ]:
# Plot delle predizioni in 3D scalate e non (internal test)

X1 = y_pred_test_scaled[:, 0] 
Y1 = y_pred_test_scaled[:, 1]  
Z1 = y_pred_test_scaled[:, 2]  

X2 = y_pred_test[:, 0]  
Y2 = y_pred_test[:, 1]  
Z2 = y_pred_test[:, 2]  


# Creazione figure 3D
fig = plt.figure(figsize=(10, 7))
ax1 = fig.add_subplot(122, projection='3d')
ax2 = fig.add_subplot(121, projection='3d')

# Scatter plot 3D
ax1.scatter(X1, Y1, Z1, c=Z1, cmap='coolwarm', marker='o')
ax2.scatter(X2, Y2, Z2, c=Z2, cmap='coolwarm', marker='o')

# Label assi
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Z")
ax1.set_title("Internal Test (StandardScaler) in 3D")

ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Z")
ax2.set_title("Internal Test in 3D")

plt.show()

In [ ]:
y_blind_test_pred_scaled= best_mlp.predict(X_blind_test_scaled)
y_blind_test_pred = scaler_y.inverse_transform(y_blind_test_pred_scaled)

In [ ]:
print(y_blind_test_pred_scaled.shape)
print(y_blind_test_pred.shape)

In [ ]:
# Plot delle predizioni in 3D scalate e non (blind test)

X1 = y_blind_test_pred_scaled[:, 0]  
Y1 = y_blind_test_pred_scaled[:, 1]  
Z1 = y_blind_test_pred_scaled[:, 2] 

X2 = y_blind_test_pred[:, 0]  
Y2 = y_blind_test_pred[:, 1]  
Z2 = y_blind_test_pred[:, 2]  


# Creazione figura 3D
fig = plt.figure(figsize=(10, 7))
ax1 = fig.add_subplot(122, projection='3d')
ax2 = fig.add_subplot(121, projection='3d')

# Scatter plot 3D
ax1.scatter(X1, Y1, Z1, c=Z1, cmap='coolwarm', marker='o')
ax2.scatter(X2, Y2, Z2, c=Z2, cmap='coolwarm', marker='o')

# Label assi
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("Z")
ax1.set_title("Blind test (StandardScaler) in 3D")

ax2.set_xlabel("X")
ax2.set_ylabel("Y")
ax2.set_zlabel("Z")
ax2.set_title("Blind test in 3D")

plt.show()

### Model Selection

In [ ]:
# Visualizzazione miglior iperparametri
best_params = grid_search.best_params_
best_params

best_params = {'activation': 'relu',
 'alpha': 0.001,
 'batch_size': 64,
 'hidden_layer_sizes': (50, 50),
 'learning_rate': 'constant',
 'learning_rate_init': 0.02,
 'momentum': 0.9,
 'solver': 'sgd'}

In [ ]:
# 5 run diverse corrispondenti a 5 random_state differenti (MODEL SELECTION)

epochs = 500

# Seed differenti
random_states = [7, 13, 26, 39, 47]

for rs in random_states:
    print(f"Training con random_state={rs}...")

    # Inizializzazione modello con i miglior iperparametri trovati
    nn = MLPRegressor(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        solver='sgd',
        alpha=best_params['alpha'],
        learning_rate_init=best_params['learning_rate_init'],
        learning_rate=best_params['learning_rate'],
        momentum=best_params['momentum'],
        batch_size=64,  # fullbatch (64)
        max_iter=1, # nota bene
        warm_start=True, # nota bene
        shuffle=True,
        random_state=rs,
        verbose=False
      )

    train_mse_list = []
    val_mse_list = []
    train_mee_list = []
    val_mee_list = []

    patience = 30  # Numero di epoche senza miglioramenti prima di fermarsi
    best_val_loss = float('inf')
    counter = 0  

    for epoch in range(epochs):  
        nn.partial_fit(X_train_scaled, y_train_scaled)
        
        # Predizioni su training e test set
        y_train_pred_scaled = nn.predict(X_train_scaled)
        y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)
        y_val_pred_scaled = nn.predict(X_val_scaled)
        y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)

        # Calcolo metriche     
        train_mse = mean_squared_error(y_train, y_train_pred)
        val_mse = mean_squared_error(y_val, y_val_pred)

        train_mee = mean_euclidean_error(y_train, y_train_pred)
        val_mee = mean_euclidean_error(y_val, y_val_pred)

        # Memorizziamo i valori di loss e accuracy
        train_mse_list.append(train_mse)    
        val_mse_list.append(val_mse)
        train_mee_list.append(train_mee)
        val_mee_list.append(val_mee)

        if val_mse < best_val_loss:
            best_val_loss = val_mse
            counter = 0
        else:
            counter += 1

        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Creazione della figura con due subplot
    fig, ax1 = plt.subplots(1, 2, figsize=(12, 5))

    # Plot MSE
    ax1[0].plot(train_mse_list, label="Train MSE", color="blue")
    ax1[0].plot(val_mse_list, label="Validation MSE", color="green")
    ax1[0].set_xlabel("Epochs")
    ax1[0].set_ylabel("MSE")
    ax1[0].set_title(f"MSE CUP (random_state={rs})")
    ax1[0].legend()
    ax1[0].grid(True)

    # Plot Accuracy
    ax1[1].plot(train_mee_list, label="Train MEE", color="blue")
    ax1[1].plot(val_mee_list, label="Validation MEE", color="green")
    ax1[1].set_xlabel("Epochs")
    ax1[1].set_ylabel("MEE")
    ax1[1].set_title(f"MEE CUP (random_state={rs})")
    ax1[1].legend()
    ax1[1].grid(True)

    # Mostra la figura completa con i due subplot
    plt.tight_layout()
    plt.show()

    # print(f'Best params: {best_params}')
    # Stampa dei risultati finali per ogni seed
    print(f"Risultati per random_state={rs}:")
    print(f"  - Train MSE: {train_mse_list[-1]:.5f}, Validation MSE: {val_mse_list[-1]:.5f}")
    print(f"  - Train MEE: {train_mee_list[-1]:.5f}, Validation MEE: {val_mee_list[-1]:.5f}")
    print("-" * 50)

### Re-Training Best Model

In [ ]:
# Visualizzazione miglior iperparametri
best_params = grid_search.best_params_
best_params

best_params = {'activation': 'relu',
 'alpha': 0.001,
 'batch_size': 64,
 'hidden_layer_sizes': (50, 50),
 'learning_rate': 'constant',
 'learning_rate_init': 0.02,
 'momentum': 0.9,
 'solver': 'sgd'}

In [ ]:
# Creazione del miglior modello con gli iperparametri trovati
best_mlp = MLPRegressor(
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    solver='sgd',
    alpha=best_params['alpha'],
    learning_rate_init=best_params['learning_rate_init'],
    learning_rate=best_params['learning_rate'],
    momentum=best_params['momentum'],
    batch_size=64,  # fullbatch (64)
    max_iter=1,  # nota bene
    warm_start=True, # nota bene
    shuffle=True,
    random_state=39,
    verbose=False,
)

In [ ]:
epochs = 500

train_mse_list = []
test_mse_list = []
train_mee_list = []
test_mee_list = [] 

for epoch in range(epochs):  
    # Allenamento sul training set
    best_mlp.partial_fit(X_train_scaled, y_train_scaled)

    # Predizioni su training e test set
    y_train_pred_scaled = best_mlp.predict(X_train_scaled) 
    y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)  
    y_test_pred_scaled = best_mlp.predict(X_test_scaled) 
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)  

    # Calcolo delle metriche
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    train_mee = mean_euclidean_error(y_train, y_train_pred)
    test_mee = mean_euclidean_error(y_test, y_test_pred)

    # Memorizziamo i valori di MSE e MEE
    train_mse_list.append(train_mse)    
    test_mse_list.append(test_mse)
    train_mee_list.append(train_mee)
    test_mee_list.append(test_mee)

In [ ]:
# Creazione della figura con due subplot
fig, ax1 = plt.subplots(1, 2, figsize=(12, 5))

# Plot MSE
ax1[0].plot(train_mse_list, label="Train MSE", color="blue")
ax1[0].plot(test_mse_list, label="Test MSE", color="red")
ax1[0].set_xlabel("Epochs")
ax1[0].set_ylabel("MSE")
ax1[0].set_title(f"MSE CUP")
ax1[0].legend()
ax1[0].grid(True)

# Plot MEE
ax1[1].plot(train_mee_list, label="Train MEE", color="blue")
ax1[1].plot(test_mee_list, label="Test MEE", color="red")
ax1[1].set_xlabel("Epochs")
ax1[1].set_ylabel("MEE")
ax1[1].set_title(f"MEE CUP")
ax1[1].legend()
ax1[1].grid(True)

# Figura completa con i due subplot
plt.tight_layout()
plt.show()

print(f"  - Train MSE: {train_mse_list[-1]:.5f}, Test MSE: {test_mse_list[-1]:.5f}")
print(f"  - Train MEE: {train_mee_list[-1]:.5f}, Test MEE: {test_mee_list[-1]:.5f}")
print("-" * 50)

In [ ]:
# Valutazione sull'Internal Test Set
y_pred_test_scaled = best_mlp.predict(X_test_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1, y_test.shape[1]))

test_mse = mean_squared_error(y_test, y_pred_test)
test_mee = mean_euclidean_error(y_test, y_pred_test)

print(f"MSE Internal Test: {test_mse}")
print(f"MEE Internal Test: {test_mee}")

In [ ]:
# Plot delle predizioni in 3D (internal test)

X = y_pred_test[:, 0]  
Y = y_pred_test[:, 1]  
Z = y_pred_test[:, 2] 

# Creazione figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot 3D
ax.scatter(X, Y, Z, c=Z, cmap='coolwarm', marker='o')

# Label assi
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Internal Test in 3D")

plt.show()

In [ ]:
y_blind_test_pred_scaled= best_mlp.predict(X_blind_test_scaled)
y_blind_test_pred = scaler_y.inverse_transform(y_blind_test_pred_scaled)

In [ ]:
print(y_blind_test_pred.shape)

In [ ]:
# Plot delle predizioni in 3D (blind test)

X = y_blind_test_pred[:, 0]  
Y = y_blind_test_pred[:, 1]  
Z = y_blind_test_pred[:, 2] 

# Creazione figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot 3D
ax.scatter(X, Y, Z, c=Z, cmap='coolwarm', marker='o')

# Label assi
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Blind Test in 3D")

plt.show()